### This script contains:

#### 1. Import libraries & datasets
#### 2. Clean the Customers dataframe
#### 3. Check for duplicates, nulls and mixed-types in Customers dataframe
#### 4. Prepare the dataframes, 4-8 and Customers, to be merged
#### 5. Merging the dataframes
#### 6. Exporting final dataframe

## Importing libraries and datasets

In [1]:
#Import libraries 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#Folder path into usable string
path = r'C:\Users\willm\Instacart Basket Analysis'

In [3]:
#Import 4.8 final Dataset
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4-8.pkl'))
cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

### Cleaning Cust dataframe

In [4]:
#Checking the column headers
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
#Dropping 'First Name' and 'Surnam' columns, as we do not need that data
cust = cust.drop(columns = ['First Name', 'Surnam'])

In [6]:
#Changing Gender to all lower case for easier typing later
cust.rename(columns = {'Gender' : 'gender'}, inplace = True)
#Changing STATE to all lower case for easier typing later
cust.rename(columns = {'STATE' : 'state'}, inplace = True)
#Changing Age to all lower case for easier typing later
cust.rename(columns = {'Age' : 'age'}, inplace = True)
#Changing n_dependants to children as it would be an easier name to remember, and write, later
cust.rename(columns = {'n_dependants' : 'children'}, inplace = True)
#Changing fam_status to f_status
cust.rename(columns = {'fam_status' : 'f_status'}, inplace = True)

In [7]:
cust.head()

,user_id,gender,state,age,date_joined,children,f_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


### Checking for duplicates, nulls and mixed-types in Cust dataframe

In [8]:
#Checking the data types for each column
#Also checking the size of the dataframe
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      206209 non-null  int64 
 1   gender       206209 non-null  object
 2   state        206209 non-null  object
 3   age          206209 non-null  int64 
 4   date_joined  206209 non-null  object
 5   children     206209 non-null  int64 
 6   f_status     206209 non-null  object
 7   income       206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


In [9]:
#Checking for empty fields in all columns
cust.isnull().sum()

user_id        0
gender         0
state          0
age            0
date_joined    0
children       0
f_status       0
income         0
dtype: int64

In [10]:
cust.head()

,user_id,gender,state,age,date_joined,children,f_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [11]:
#Checking for duplicates
cust_dup = cust[cust.duplicated()]

In [12]:
cust_dup

,user_id,gender,state,age,date_joined,children,f_status,income


In [13]:
#No Duplicates found

In [14]:
#Checking for mixed types in columns
for col in cust.columns.tolist():
    weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (cust[weird]) > 0:
        print (col)

In [15]:
#No mix types found

### Preparing the dataframes, 4-8 and Customers, to be merged

#### Checking the data types and columns for the merge

In [16]:
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,_merge,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,spend_group,spend_flag,days_since_last_order_median,order_frequency_flag
0,2539329,1,1,2,8,NaN,First Order,196,1,0,...,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.26,Non-frequent customer
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,1,...,both,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.26,Non-frequent customer
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,1,...,both,Mid-range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.26,Non-frequent customer
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,1,...,both,Mid-range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.26,Non-frequent customer
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,1,...,both,Mid-range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.26,Non-frequent customer


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32399732 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                        Dtype   
---  ------                        -----   
 0   order_id                      object  
 1   user_id                       int64   
 2   order_number                  int64   
 3   orders_day_of_week            int64   
 4   order_hour_of_day             int64   
 5   days_since_prior_order        float64 
 6   first_order                   object  
 7   product_id                    int64   
 8   add_to_cart_order             int64   
 9   reordered                     int64   
 10  product_name                  object  
 11  aisle_id                      int64   
 12  department_id                 int64   
 13  prices                        float64 
 14  _merge                        category
 15  price_range_loc               object  
 16  busiest_day                   object  
 17  busiest_period_of_day         object  
 18  

In [18]:
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      206209 non-null  int64 
 1   gender       206209 non-null  object
 2   state        206209 non-null  object
 3   age          206209 non-null  int64 
 4   date_joined  206209 non-null  object
 5   children     206209 non-null  int64 
 6   f_status     206209 non-null  object
 7   income       206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


In [19]:
#Dropping the _merge column - to allow for next merge - and add to cart order column
df = df.drop(columns = ['_merge', 'add_to_cart_order'])

### Changing data types for several columns, to allow faster merging.

In [20]:
#Changing data types
df['user_id'] = df['user_id'].astype('str')
df['order_number'] = df['order_number'].astype('int8')
df['orders_day_of_week'] = df['orders_day_of_week'].astype('int8')
df['order_hour_of_day'] = df['order_hour_of_day'].astype('int8')
df['product_id'] = df['product_id'].astype('int32')
df['reordered'] = df['reordered'].astype('int8')
df['aisle_id'] = df['aisle_id'].astype('int8')


In [21]:
df['department_id'] = df['department_id'].astype('int8')
df['max_order'] = df['max_order'].astype('int8')
df['days_since_prior_order'] = df['days_since_prior_order'].astype('float16')
df['prices'] = df['prices'].astype('float16')
df['spend_group'] = df['spend_group'].astype('float16')
df['days_since_last_order_median'] = df['days_since_last_order_median'].astype('float16')

In [22]:
#Changing data types
cust['user_id'] = cust['user_id'].astype('str')
cust['children'] = cust['children'].astype('int8')
cust['age'] = cust['age'].astype('int8')
cust['income'] = cust['income'].astype('int32')

In [23]:
#Comfirming data type changes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32399732 entries, 0 to 32404858
Data columns (total 22 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   order_id                      object 
 1   user_id                       object 
 2   order_number                  int8   
 3   orders_day_of_week            int8   
 4   order_hour_of_day             int8   
 5   days_since_prior_order        float16
 6   first_order                   object 
 7   product_id                    int32  
 8   reordered                     int8   
 9   product_name                  object 
 10  aisle_id                      int8   
 11  department_id                 int8   
 12  prices                        float16
 13  price_range_loc               object 
 14  busiest_day                   object 
 15  busiest_period_of_day         object 
 16  max_order                     int8   
 17  loyalty_flag                  object 
 18  spend_group         

In [24]:
#Comfirming data type changes
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      206209 non-null  object
 1   gender       206209 non-null  object
 2   state        206209 non-null  object
 3   age          206209 non-null  int8  
 4   date_joined  206209 non-null  object
 5   children     206209 non-null  int8  
 6   f_status     206209 non-null  object
 7   income       206209 non-null  int32 
dtypes: int32(1), int8(2), object(5)
memory usage: 9.0+ MB


### Merging the dataframes. With a time check.

In [25]:
%%time
df_combined = df.merge(cust, on = ['user_id'], indicator = True)

Wall time: 2min 58s


In [26]:
df_combined.head(30)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,reordered,product_name,...,days_since_last_order_median,order_frequency_flag,gender,state,age,date_joined,children,f_status,income,_merge
0,2539329,1,1,2,8,NaN,First Order,196,0,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,Repeat Customer,196,1,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,Repeat Customer,196,1,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,Repeat Customer,196,1,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,Repeat Customer,196,1,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
5,3367565,1,6,2,7,19.0,Repeat Customer,196,1,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
6,550135,1,7,1,9,20.0,Repeat Customer,196,1,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
7,3108588,1,8,1,14,14.0,Repeat Customer,196,1,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
8,2295261,1,9,1,16,0.0,Repeat Customer,196,1,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
9,2550362,1,10,4,8,30.0,Repeat Customer,196,1,Soda,...,20.265625,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both


In [27]:
df_combined.tail(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,reordered,product_name,...,days_since_last_order_median,order_frequency_flag,gender,state,age,date_joined,children,f_status,income,_merge
32399722,1539810,106143,21,1,18,5.0,Repeat Customer,19675,1,Organic Raspberry Black Tea,...,7.640625,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,both
32399723,3308056,106143,22,4,20,10.0,Repeat Customer,19675,1,Organic Raspberry Black Tea,...,7.640625,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,both
32399724,2988973,106143,23,2,22,5.0,Repeat Customer,19675,1,Organic Raspberry Black Tea,...,7.640625,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,both
32399725,930,106143,24,6,12,4.0,Repeat Customer,19675,1,Organic Raspberry Black Tea,...,7.640625,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,both
32399726,467253,106143,25,6,16,7.0,Repeat Customer,19675,1,Organic Raspberry Black Tea,...,7.640625,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,both
32399727,156685,106143,26,4,23,5.0,Repeat Customer,19675,1,Organic Raspberry Black Tea,...,7.640625,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,both
32399728,484769,66343,1,6,11,NaN,First Order,47210,0,Fresh Farmed Tilapia Fillet,...,26.328125,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151,both
32399729,1561557,66343,2,1,11,30.0,Repeat Customer,47210,1,Fresh Farmed Tilapia Fillet,...,26.328125,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151,both
32399730,276317,66343,3,6,15,19.0,Repeat Customer,47210,1,Fresh Farmed Tilapia Fillet,...,26.328125,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151,both
32399731,2922475,66343,4,1,12,30.0,Repeat Customer,47210,1,Fresh Farmed Tilapia Fillet,...,26.328125,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151,both


In [28]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32399732 entries, 0 to 32399731
Data columns (total 30 columns):
 #   Column                        Dtype   
---  ------                        -----   
 0   order_id                      object  
 1   user_id                       object  
 2   order_number                  int8    
 3   orders_day_of_week            int8    
 4   order_hour_of_day             int8    
 5   days_since_prior_order        float16 
 6   first_order                   object  
 7   product_id                    int32   
 8   reordered                     int8    
 9   product_name                  object  
 10  aisle_id                      int8    
 11  department_id                 int8    
 12  prices                        float16 
 13  price_range_loc               object  
 14  busiest_day                   object  
 15  busiest_period_of_day         object  
 16  max_order                     int8    
 17  loyalty_flag                  object  
 18  

### Exporting final dataframe

In [29]:
df_combined.to_pickle(os.path.join(path, '02 Data','Prepared Data', '49_df_combined.pkl'))